# AS09p Youtube comment clustering

在AS06萃取關鍵字的練習中，我們曾使用反送中相關影片之留言共八千則作為範例。這次的作業將邀請你使用該筆資料，先篩檢關鍵字，再透過群集法，嘗試找出留言有哪些主題、立場、或者網民討論的面向等等。這次的作業是開放式的結果，也就是每個人所找出來的主題或立場等可能不太一樣，但目標是一致的，找出這些文本裡面有哪些主題、在討論什麼。這種問問題的方式其實也出現在不少專題組別中。你在寫作業的過程，應該會不斷地感受到，這怎麼抽得出來主題？這真的抽得出來嗎？這樣抽出來的主題有效嗎？有意義嗎？然後為了獲得能說服自己的主題，你可能會不斷反覆篩選這些留言，卻又會遇到對代表性的質疑，也就是，縮減到這麼小的資料集合，做出來的結果會有代表性嗎？

所以這個作業有個目的是先讓你透過做作業了解到，如果你做的專題是想回答「這些文本裡面有哪些主題」，很可能會遇到什麼樣的問題。

本週的作業程式碼無特殊規定，要怎麼處理這些資料，或者要怎麼解釋跑出來的東西，都讓同學自由發揮。唯獨需要回答你做了什麼樣的篩檢，還有繪製出視覺化的群集結果，和自我評估用群集來找主題是否有效。

## (TODO) Answer the following questions

### (TODO)Document filtering strategies
目前作業給的資料一共有8000筆留言，每筆留言的欄位有發佈時間、更新時間、按讚數、也可以計算留言的字數，請問你採用了什麼樣的篩選策略？請於以下的文字框回答（編輯下方Markdown，需說明至少五個連續處理過程）。這種過程的書寫通常會出現在論文或者研究報告中，或者出現在學術海報的一小塊paragraph。

（範例答案）
1. 原資料有8000筆留言
2. 篩除XX數不滿XX的留言後，剩下XXX筆留言
3. 篩除XX後不滿XX後，剩下X筆留言
4. ...
5. ...

### (TODO)Vocabulary filtering strategies
在教學範例中，我們介紹不少Strategy來篩選不必要或必要的文字，請問你怎麼做關鍵字篩選？

1. 原始資料在斷詞後共有XXX個相異字詞。（規定填寫）
2. 在經過篩除標點符號後，剩下XXX個相異字詞。（規定填寫）
3. 再經過XXX後，剩下XXX個相異字詞。
4. ...
5. ...

## Loading youtube data

### by colab

In [ ]:
!wget https://github.com/P4CSS/PSS/blob/master/data/yt-comment-antiELAB.xlsx?raw=true -O yt-comment-antiELAB.xlsx
import pandas as pd
df = pd.read_excel('yt-comment-antiELAB.xlsx') 
df

### by local notebook

In [7]:
import pandas as pd
df = pd.read_excel('../data/yt-comment-antiELAB.xlsx')
df

,videoId,commentId,parentId,authorDisplayName,textOriginal,likeCount,publishedAt,updatedAt
0,2_tXjqhbe5E,UgwQIF9qNqGydjE2NkN4AaABAg,NaN,CHI-HAU CHEN,鄭大哥和吉雷米的互動很可愛，歡迎加入台灣這個溫馨的家庭喔~~~~,19,2020-04-02T01:40:29Z,2020-04-02T01:40:29Z
1,2_tXjqhbe5E,UgwuP0Jva-U69xTMaEF4AaABAg,NaN,Funky Duck,牛嘴掩.....\n這高雄老外的台語能力應該比台北人好,1,2020-04-07T04:36:37Z,2020-04-07T04:36:37Z
2,2_tXjqhbe5E,UgyVlhx36V2XWLOM9MZ4AaABAg,NaN,Ally,Zoom把客戶資料洩給中國，現在紐約市已不讓學生用這系統視訊上課了!,0,2020-04-06T19:54:04Z,2020-04-06T19:54:04Z
3,2_tXjqhbe5E,Ugz4v7OudQxaDXYyegZ4AaABAg,NaN,Ally,法國人執行居家避疫比起美國比較確實，外出還要有通行證,0,2020-04-06T19:18:24Z,2020-04-06T19:18:24Z
4,2_tXjqhbe5E,Ugyb-ogACbZWVewnU-94AaABAg,NaN,Kitty Wong,仆街鄭,0,2020-04-06T15:20:34Z,2020-04-06T15:20:34Z
...,...,...,...,...,...,...,...,...
7995,ySMAcMAL6rY,UgzU9oD5I6q1qHLzx_B4AaABAg,NaN,nova lee,自己不是會算嗎,1,2019-08-22T04:00:31Z,2019-08-22T04:00:31Z
7996,zHlhQoT9OF0,Ugz2ff-Be0yCoEUY-Rd4AaABAg,NaN,the world Rock,常德說的真好,3,2020-04-10T03:21:34Z,2020-04-10T03:21:34Z
7997,zQWzh4yj_g8,Ugwex9I2lZWa-DgeWrx4AaABAg,NaN,Zoe Su,自私的人多的是，水準就和中國人一樣,1,2020-02-12T22:13:24Z,2020-02-12T22:13:24Z
7998,zQWzh4yj_g8,UgxpokMr9hHK0Ugcvl94AaABAg,NaN,非也非也,人都自私自利的，適者生存，不適者自己想辦法，沒有人會幫的，靠自己最實在,0,2020-02-12T04:53:28Z,2020-02-12T04:53:28Z


## Feature selections - selecting Model

## Clustering

### Evaluating

### Final clustering by k=?

## Visualization

視覺化是必要選項。

## (TODO) Explaining your results
從上述群集的結果，你觀察到有哪些留言群集？請評估你所找出來的關鍵字群集。

(Your Answer here)
在前述篩選策略下，從Evaluating的結果，我選擇以k=?作為群集數。
經視覺化每個群集的關鍵字後，一共發現以下幾大主題。
1. k=0: 該群集為...
2. k=1: 該群集的主要關鍵字為...屬於...主題
3. k=2: 
4. k=3: 
5. k=?...

有效的主題我認為共有「XXXX（k=0, 1」、「XXXXX（k=2）」、「XXXXX（k=3, 4）」、「」等n個主題。`k=5`的群集只有單一關鍵字極高，其他關鍵字稀少，可能是受到......的影響。將包含關鍵字相關的留言抽出結果如下方dataFrame所示，為...的主題。

## Final Modification (if you have)
經觀察前述群集結果後，我篩除了XXX關鍵字，重新進行群集後所得到的結果為以下幾個主題，分別包含「XXXX（k=0, 1」、「XXXXX（k=2）」、「XXXXX（k=3, 4）」、「」
